In [1]:
import numpy as np
import h5py
import seaborn
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import multiprocessing as mp
import glob
import sys
from functools import partial
from sklearn.cluster import DBSCAN
import matplotlib as mpl
import collections
from collections import defaultdict
from skimage.filters import scharr

In [23]:
def Edge_Detection(image,threshold=99.5,image_threshold=97,radius=15,show=True,select=3,imagename='Nil'):
    '''This function uses the skimage edge detection filters to find the PIL'''
    image = Edge_Filter(image,threshold=image_threshold)
    edge = scharr(image)
    pixel_coor = []
    pixel_weight = []
    threshold = np.percentile(edge,threshold)
    
    for i in range(edge.shape[0]):
        for j in range(edge.shape[1]):
            if edge[i,j]<threshold:
                edge[i,j]=0
            else:
                pixel_coor.append(np.array([i,j]))
                pixel_weight.append(edge[i,j]) # the edge detection magnitude
    
    pixel_coor, pixel_weight = np.array(pixel_coor), np.array(pixel_weight)
    # now we have the key pixels of interest stored in pixel_coor and pixel_weight
    # now we do the clustering
    clust = DBSCAN(eps=radius,min_samples=5)
    thecluster = clust.fit_predict(X=pixel_coor)
    
    # now we are going to select the top few clusters with the largest sum of xy-gradient
    N = len(set(thecluster))
    cluster_coor = defaultdict(list)
    cluster_gradient = defaultdict(list)
    whole_data = list(zip(pixel_coor,pixel_weight,thecluster))
    new_data = []
    
    if (-1) in thecluster:
        N = N-1
        for item in whole_data:
            if item[2]!=-1:
                new_data.append(item)
    
    for c in range(N):
        cluster_coor[c] = [item[0] for item in new_data if item[2]==c]
        cluster_gradient[c] = [item[1] for item in new_data if item[2]==c]
        
    cluster_sum = [sum(cluster_gradient[c]) for c in cluster_gradient]
    if len(cluster_sum)>select:
        top = np.argsort(cluster_sum)[-(select):] 
    else:
        top = list(range(len(cluster_sum)))
        
    PIL_pixel = []
    
    for c in top:
        coor = cluster_coor[c]
        for point in coor:
            PIL_pixel.append(point)
    
    PIL_pixel = np.array(PIL_pixel)
                
    if show==True:
        N = len(set(thecluster))
        cmap = plt.cm.jet
        cmaplist = [cmap(i) for i in range(cmap.N)]
        cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)
        bounds = np.linspace(-1,N,N+2)
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        
        fig = plt.subplots(nrows=1, ncols=2)

        plt.subplot(1, 2, 1)
        seaborn.heatmap(image, center=0, cbar=False,xticklabels=False, yticklabels=False)
            
        plt.subplot(1, 2, 2)
        plt.scatter(pixel_coor[:, 1], pixel_coor[:, 0], c=thecluster,cmap=cmap, marker='o', s=0.5)
        plt.xlim(0, image.shape[1])
        plt.ylim(0, image.shape[0])
        plt.gca().invert_yaxis()
        plt.tick_params(labelbottom='off')
        plt.tick_params(labelleft='off')

        plt.show()
    elif show=='select':
        fig = plt.subplots(nrows=1, ncols=2)

        plt.subplot(1, 2, 1)
        seaborn.heatmap(image, center=0, cbar=False,xticklabels=False, yticklabels=False)
            
        plt.subplot(1, 2, 2)
        plt.scatter(PIL_pixel[:, 1], PIL_pixel[:, 0], color='red',marker='o', s=0.05)
        plt.xlim(0, image.shape[1])
        plt.ylim(0, image.shape[0])
        plt.gca().invert_yaxis()
        plt.tick_params(labelbottom='off')
        plt.tick_params(labelleft='off')
        plt.savefig(imagename+'_s_f'+str(select)+'.pdf')
        plt.show()
    else:
        return PIL_pixel

In [93]:
class PIL:
    '''This class creates a PIL detector which can be called for fitting PIL on any Bz image or an entire hdf5 file.
        
       Summary on the class "PIL":
       
       For each image of the vertical component of the magnetic field, namely the image of "Bz", we want to find the 
       contour of Bz==0 that separates the strong positive magnetic polars and the associated negative polars, which 
       is called the Polarity Inversion Line(PIL). Physicists are interested in the local magnetic field features 
       around the PIL.
       
       In a pixelized image, the pixels that consist of the PIL should be those pixels that have a large Bz gradient in 
       its neighborhood. And finding these points is analogous to the edge detection task in the image processing field.
       So by applying an edge detection filter upon each Bz image, we could find the candidate pixels of the PIL. And by 
       further applying a clustering algorithm on the candidate pixels, we are able to locate several PIL segments in the
       image. Finally, one could retain, as many as one wanted, the PILs with large average gradients for future purposes.
       
       The class PIL is a parameterized PIL detector that combines the data preparation, edge detection, clustering and 
       parallelized training, and visualization procedures. It can be applied to a piece of image in numpy ndarray form, 
       and even an entire hdf5 file with rather fast implementation.
    
    '''
    
    def __init__(self,edge_retain=0.005,polar_threshold=0.50,radius=12,min_samples=5):
        '''Initialize the PIL detector
        
           Params:
           edge_retain: The fraction of candidate PIL pixels of the input image. Range from 0 to 1. Float number.
                        What fraction of the pixels of the whole input image can be considered as a candidate for PIL. 
                        The larger the amount is, the more pixels there will be for the PIL. The default amount is 0.005
                        which corresponds to 0.5% of all pixels in the input image. These pixels are the pixels with the 
                        largest local gradient. By specifying the edge_retain to be too large, we may include many irrelevant
                        pixels when drawing the PIL. By setting the parameter to be too small, we may lose some pixels on
                        the true PIL. The best way to tune the parameter is to use the .visualize() method to check visually. 
                        Generally speaking, each image won't have more than 1% of its pixels as PIL.
                        
           polar_threshold: The strong polar threshold. Range from 0 to 1. Float number.
                            The quantile of the magnitude of the vertical component above which a pixel is considered as 
                            a strong polar. Basically, each image is decomposed into a positive image and a complementing
                            negative image. In a positive image, all negative image pixels are coerced to 0, and vice versa.
                            In the positive image, for instance, all pixels' values are ranked from the smallest to the 
                            largest, and all the pixels that are ranking at the quantile no greater than polar_threshold are
                            further coerced to 0. The same procedure applies to the negative image. And the positive and 
                            negative image are put back together as a sparser image, upon which the edge detector will be
                            applied. Setting the parameter too small can lead to many weaker polars being considered when
                            drawing the PIL.
                            
           radius: The maximum pixel distance between two pixels that make the pixels being considered as on the same PIL. Positive Integer. 
                   The larger the quantity, the less PIL segments will be drawn in the image, but maybe two originally 
                   separated PILs will be considered mistakenly as a single PIL.
           
           min_samples: The minimum number of pixels that a PIL segment should contain. Positive Integer.
                        For a set of pixels to be considered as a PIL, one need at least min_samples in the set. By setting
                        the parameter to be too large, we may mistakenly overlook those shorter PIL. On the contrast, we may
                        include too many noisy PILs that are not ideal.
        '''
        
        self.edge_retain = edge_retain
        self.polar_threshold = polar_threshold*100
        self.radius = radius
        self.min_samples = min_samples
        self.DBSCAN = DBSCAN(eps=self.radius,min_samples=self.min_samples) # set up the DBSCAN cluster operator for pixels clustering
        self.image = None
        self.file = None
        self.PIL_segment = dict()
        self.global_mode = False
        
    def Image_Decompose(self,image):
        '''Data preparation method, a submethod for .data_preparation() method.
           
           Params:
           image: Input image. Numpy array.
                  Shape = [height,width]
           
           Output:
           pos_weight: Numpy ndarray.
                       Shape = [num_positive_pixels,]
                       
                       An array of all positive pixel values of the input image.
                       
           neg_weight: Numpy ndarray.
                       Shape = [num_negative_pixels,]
                       
                       An array of all negative pixel values of the input image.
        '''
        image = np.nan_to_num(image) # fill the NaN value in an image as 0
        
        pos_weight = np.where(image>1e-6,image,0) # create a positive image component
        neg_weight = np.where(image<-1e-6,image,0) # create a negative image component
        
        pos_weight, neg_weight = np.ndarray.flatten(pos_weight), np.ndarray.flatten(neg_weight) # collapse both image into 1-D
        
        pos_weight = pos_weight[np.nonzero(pos_weight)] # remove all 0s in the pos_weight
        neg_weight = neg_weight[np.nonzero(neg_weight)] # remove all 0s in the neg_weight
    
        
        return pos_weight, neg_weight
    
    def data_preparation(self,image,window_size=5):
        '''Data preparation method, a submethod for the .fit() and .fitFile() method.
        
           Params:
           image: Input image. Numpy array.
                  Shape = [height,width]
           
           window_size: The range of a pixel's neighborhood. Odd number integer. No smaller than 3.
                        For each pixel in the input image, we consider a window_size*window_size, 5*5 by default, subimage with
                        the pixel of interest being at the center. 
                        
           Output:
           image_new: Output image ready for edge detection. Numpy array.
                      Shape = [height, width], same as the input image.
                      
                      For each pixel in the original image, one consider the local maximum and local minimum of the pixel within
                      its neighborhood. If both the local maximum and minimum passed the quantile threshold sepecified by 
                      self.polar_threshold in the corresponding positive and negative image, the pixel's value is retained,
                      otherwise the pixel is put to zero.
                      
                      As a result, the output image only contains those pixels with both a strong positive and a strong negative
                      polar within its neighborhood. This helps exclude those pixels with a strong positive/negative polar but 
                      no associative polar of the opposite polarity. So these pixels in the output image is a superset of PIL pixels.
        '''
        
        image = np.nan_to_num(image) # fill all NaN in the image as 0
        self.window_size = window_size # store the window_size attribute
        
        pos_weight, neg_weight = self.Image_Decompose(image)
    
        height = image.shape[0]
        width = image.shape[1]
            
        pos_threshold = np.percentile(pos_weight, q=self.polar_threshold) # find the top quantile value of all positive pixels
        neg_threshold = np.percentile(np.abs(neg_weight), q=self.polar_threshold) # find the top quantile value of all negative pixels
    
        pad_size = int((window_size-1)/2)
        image_padded = np.pad(image,((pad_size,pad_size),(pad_size,pad_size)),'constant') # pad the image to make sure the output image has the same shape as the input image
    
        image_max = np.zeros((height,width)) # an image with each pixel storing its local maximum
        image_min = np.zeros((height,width)) # an image with each pixel storing its local minimum
        image_new = np.zeros((height,width)) # output image
    
    
        for i in range(height):
            for j in range(width):
                x = i+pad_size
                y = j+pad_size
            
                window = image_padded[(x-pad_size):(x+pad_size+1),(y-pad_size):(y+pad_size+1)] # the neighborhood subimage for each pixel
                maximum = np.amax(window) # find the local maximum
                minimum = np.amin(window) # find the local minimum
            
                image_max[i,j] = maximum
                image_min[i,j] = minimum
                if maximum>pos_threshold and minimum<-(neg_threshold): # retains the pixel value only if it has both a strong positive polar and a strong negative polar in its neighborhood subimage
                    image_new[i,j] = image[i,j]
                
        return image_new
    
    def fit(self,image,select=1,window_size=5):
        ''' Fit the whole PIL pipeline on the input image. Including the edge detection, clustering and filtering steps. Given
            an input image, the fit method find all PIL pixels, and return two arrays containing information of the selected 
            PILs with the largest average local magnetic field gradient. Each PIL's information includes its corresponding pixels
            and average local gradients.
            
            Params:
            image: Input image. Numpy array.
                   Shape = [height,width]
            
                   Original image upon which one wants to locate several PILs.
            
            select: Number of PILs to record. Positive Integer. Default is 1.
                    
                    Out of all PIL segments detected, how many of them are going to be recorded. All PIL segments are ranked 
                    according to their pixel-average local magnetic gradient, and only the PILs with the largest average gradient
                    are recorded.
            
            window_size: The range of a pixel's neighborhood. Odd number integer. No smaller than 3.
                         See description in method .data_preparation()
            
            Output: 
            PIL_segment: Information of the pixels of the selected PIL segments. Dictionary.
                       
                       The output is a dictionary: PIL_segment[segment_label]['coor'] = array of the coordinates of all pixels of the segment 
                                                   PIL_segment[segment_label]['gradient'] = average gradient of the PIL segment
                                                   
                       PIL segment labels are ranked in a descending order based on the average gradient.
        '''
        
        image = self.data_preparation(image,window_size=window_size) # only retain the pixels with both a strong positive and a strong negative polar in the neighborhood
        edge = scharr(image) # edge detection 
        
        threshold = np.percentile(edge,q=100*(1-self.edge_retain)) # calculates the top quantile for local gradients
    
        # take records of the coordinates of all pixels that have a very large local graidents.
        edge = np.where(edge>threshold,edge,0)
        pixel_coor = np.transpose(np.nonzero(edge)) # these are the PIL candidates' coordinates
        pixel_weight = list(map(lambda x: edge[x[0],x[1]],pixel_coor)) # the corresponding local gradient values for the PIL candidates
        
        thecluster = self.DBSCAN.fit_predict(X=pixel_coor) # cluster the PIL candidates into possibly several PIL segments.
    
        # now we are going to select the top few clusters with the largest average gradient
        N = len(set(thecluster)) # number of PIL segments
        cluster_coor = defaultdict(list) # a dictionary: cluster_coor[PIL_segment_label] = list of pixels of the PIL segment
        cluster_gradient = defaultdict(list) # a dictionary: cluster_gradient[PIL_segment_label] = list of pixels' gradient of the PIL segment
        
        whole_data = list(zip(pixel_coor,pixel_weight,thecluster))
        new_data = []
        
        # since there might be some PIL candidate pixels that do not even form a PIL segment, we should exclude all of these noisy points
        if (-1) in thecluster:
            N = N-1 
        for item in whole_data:
            if item[2]!=-1:
                new_data.append(item)
                    
        if N!=0: # if all candidate points are noises, then do nothing

            # record the coordinates information and gradient information for each of the PIL segment, namely each cluster
            for c in range(N):
                cluster_coor[c] = [item[0] for item in new_data if item[2]==c]
                cluster_gradient[c] = [item[1] for item in new_data if item[2]==c]
        
            # calculate the segment average gradient and rank all PILs, select the top few PIL segments for final output
            cluster_ave_gradient = [np.average(np.array(cluster_gradient[c])) for c in cluster_gradient]
        
            if len(cluster_ave_gradient)>select:
                top = np.argsort(cluster_ave_gradient)[-(select):] 
            else:
                top = np.argsort(cluster_ave_gradient)
        
            # record the coordinate information for the selected PIL segments
            PIL_segment = defaultdict(dict)
            order = list(reversed(top))
        
            for c in range(len(top)):
                k = order[c]
                coor = cluster_coor[k] # the list of coordinate of one of the top PIL segment
                PIL_segment[c]['coor'] = np.array(coor)
                PIL_segment[c]['weight'] = cluster_ave_gradient[k]
    
            self.PIL_segment = PIL_segment # PIL results
            self.N = N # total PIL segments found
            
        elif self.global_mode==False:
            print("No PIL in the image!")
    
    def fitFile(self,f,core=8,global_threshold=0.3,select=1,window_size=5):
        ''' Fit the PIL pipeline over all images in a numpy array file f/ hdf5 file f. Codes are made to be parallelized.
        
            Params:
            f: Input file. A file object.     
               The input file for the whole HARP region, could be a numpy .npy file, or a .hdf5 file.
               For npy file, its shape should be [num_frames,height,width]
               For hdf5 file, its directory structure should be 'video0'->'frameXXX'->'channels'-> the Bz Image
               
            core: The number of processors upon which the code is run. Default is 8.
            
            global_threshold: The percentile threshold used for selecting PILs throughout the whole video. Range from 0 to 1.
                              When each image of the file has its PILs founded, all PILs throughout the video are compared
                              based on their corresponding average gradient. And the quantile of the average gradient at
                              the global_threshold is used to delete those weak PILs that have an average gradient that is
                              below the quantile. The smaller it is, the more PILs will be retained for each image.
            
            select, window_size: See .fit() method for reference.
            
            Output:
            PIL_file: A file of the same type as the input file with the record of all the PILs left. Saved in the same folder.                      
        '''
        self.global_mode = True
        self.core = core
        pool = mp.Pool(processes=self.core)
        
        process_input = [] # multiprocess input list waiting to be fed into the multithread program
        
        if type(f)==np.ndarray: # check the type of file
            for i in range(f.shape[0]): # create an input list for the parallel workers
                process_input.append((f[i],select,window_size))
                
            # run and record PILs found for each image
            result = pool.starmap(self.fit,process_input) # this is a list of PIL segments result
                
            return result
    
    

In [81]:
data.shape

(1247, 377, 744)

In [89]:
data = np.load('HARP377.npy')
test_data = data[0:10]

In [90]:
test_data.shape

(10, 377, 744)

In [35]:
image = data[1110,:,:]

In [94]:
tool = PIL(edge_retain=0.003,polar_threshold=0.50,radius=12,min_samples=5)

In [71]:
tool.fit(image,select=1)

In [ ]:
result = tool.fitFile(test_data)

In [24]:
image = data[800,:,:]
result = Edge_Detection(image,threshold=99.7,image_threshold=50,radius=12,show=False,select=2,imagename='image800')

In [26]:
result.shape

(679, 2)